In [28]:
import tensorflow as tf

In [29]:
daily_sales_number = [21, 22, -108, 31, -1, 32, 34, 31]

In [30]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_number)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [31]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [32]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [33]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset:
    print(sales.numpy())

21
22
31
32
34
31


In [34]:
tf_dataset = tf_dataset.map(lambda x: x*130)
for sales in tf_dataset:
    print(sales.numpy())

2730
2860
4030
4160
4420
4030


In [35]:
tf_dataset = tf_dataset.shuffle(10)
for sales in tf_dataset:
    print(sales.numpy())

4030
4030
2730
4420
2860
4160


In [36]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[4030 4420]
[4030 2730]
[4160 2860]


In [37]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_number)
tf_dataset = tf_dataset.filter(lambda x: x > 0).map(lambda y: y * 130).shuffle(10).batch(2)

In [38]:
for sales in tf_dataset:
    print(sales.numpy())

[4420 4160]
[4030 2730]
[4030 2860]


In [40]:
images_ds = tf.data.Dataset.list_files("datasets/images/*/*", shuffle=False)
for file in images_ds.take(4):
    print(file.numpy())

b'datasets\\images\\cats\\1.jpeg'
b'datasets\\images\\cats\\2.jpeg'
b'datasets\\images\\cats\\3.jpeg'
b'datasets\\images\\cats\\4.jpeg'


In [43]:
images_ds = images_ds.shuffle(243)
for file in images_ds.take(4):
    print(file.numpy())

b'datasets\\images\\dogs\\images (19).jpeg'
b'datasets\\images\\dogs\\images (66).jpeg'
b'datasets\\images\\cats\\download (27).jpeg'
b'datasets\\images\\dogs\\images (25).jpeg'


In [44]:
class_names = ["cat", "dog"]

In [47]:
image_count = len(images_ds)
image_count

242

In [48]:
train_size = int(image_count * 0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [49]:
len(train_ds)

193

In [50]:
len(test_ds)

49

In [52]:
s = 'datasets\\images\\dogs\\images (19).jpeg'
s.split('\\')[-2]

'dogs'

In [56]:
import os
def get_label(file_path):
    return tf.strings.split(file_path, os.path.sep)[-2]
    # file_path.split('\\')[-2]

In [60]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [57]:
for t in train_ds.take(4):
    print(t.numpy())

b'datasets\\images\\dogs\\download (11).jpeg'
b'datasets\\images\\cats\\images - 2024-06-25T184052.986.jpeg'
b'datasets\\images\\cats\\download (31).jpeg'
b'datasets\\images\\cats\\images (40).jpeg'


In [66]:
train_ds = train_ds.map(process_image)
for image, label in train_ds.take(1):
    print(image)
    print(label)

tf.Tensor(
[[[ 80.00316    94.00316    59.00316  ]
  [ 91.42354   105.42354    70.42354  ]
  [ 91.88248   105.77701    73.093414 ]
  ...
  [ 12.         13.          5.       ]
  [ 13.953125   14.953125    6.953125 ]
  [ 15.309341   16.309341    8.309341 ]]

 [[ 77.17911    89.526764   56.179108 ]
  [ 84.77452    97.12218    63.77452  ]
  [ 86.70212    99.002396   67.796875 ]
  ...
  [ 11.449219   12.449219    4.4492188]
  [ 12.5859375  13.5859375   5.5859375]
  [ 14.449219   15.449219    7.4492188]]

 [[ 77.88689    88.88689    56.886887 ]
  [ 77.45854    88.45854    56.45854  ]
  [ 79.68582    90.68582    60.68582  ]
  ...
  [  9.985214   10.985214    2.9852142]
  [ 11.21875    12.21875     4.21875  ]
  [ 12.511887   13.511887    5.5118866]]

 ...

 [[155.27734   129.51953    98.98047  ]
  [111.60307    90.0562     59.209045 ]
  [150.56895   133.56895   105.73302  ]
  ...
  [207.77596   206.86664   152.64705  ]
  [220.69733   218.72467   170.16708  ]
  [218.6967    215.86076   172.67

In [67]:
def scale(image, label):
    return image/255, label

In [68]:
train_ds = train_ds.map(scale)

In [71]:
for image, label in train_ds.take(3):
    print("Image: ",image[0][0].numpy())
    print("Label: ", label.numpy())

Image:  [0.93333334 0.93333334 0.93333334]
Label:  b'cats'
Image:  [0.09762423 0.15397757 0.09800971]
Label:  b'dogs'
Image:  [0.17275007 0.22765204 0.        ]
Label:  b'dogs'
